In [23]:
def request_view(response):
    import webbrowser
    request_url = response.url
    base_url = '<head><base href="%s">'%(request_url)
    base_url = base_url.encode()
    content = response.content.replace(b"<head>", base_url)
    tem_html = open("tmp.html", "wb")
    tem_html.write(content)
    tem_html.close()
    webbrowser.open_new_tab("tmp.html")

In [ ]:
import requests
response = requests.get("http://m.qiubaichengren.net/")
request_view(response)
# print (response.status_code, response.content.decode())

In [ ]:
from lxml import etree
html = etree.HTML(response.content)
# html.xpath("//div[@class='page']//a[contains(text(),'下一页')]/@href")
html.xpath("//li[@class='next']/i/a/@href")
# from bs4 import BeautifulSoup
# soup = BeautifulSoup(response.content, 'html.parser')
# print(soup.prettify())

In [ ]:
import requests
from time import sleep
host = "http://m.qiubaichengren.net/"
def test_all_links(url):
    response = requests.get(url)
    print(url, response.status_code)
    from lxml import etree
    html = etree.HTML(response.content)
    links = html.xpath("//li[@class='next']/i/a/@href")
    if len(links) == 0:
        pass
    else:
        sleep(1)
        test_all_links(host + links[0])
        
test_all_links("http://m.qiubaichengren.net/")

In [ ]:
import requests
from lxml import etree

class Login(object):
    def __init__(self):
        self.headers = {
            'Referer': 'https://github.com/',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
            'Host': 'github.com'
        }
        self.login_url = 'https://github.com/login'
        self.post_url = 'https://github.com/session'
        self.logined_url = 'https://github.com/settings/profile'
        self.session = requests.Session()

    def token(self):
        response = self.session.get(self.login_url, headers = self.headers)
        selector = etree.HTML(response.text)
        token = selector.xpath('//*[@id="login"]/form/input[2]/@value')
        return token
    
    def login(self, email, password):
        post_data = {
            'commit': 'Sign in',
            'utf8': '✓',
            'authenticity_token': self.token(),
            'login': email,
            'password': password
        }
        response = self.session.post(self.post_url, data=post_data, headers=self.headers)
        print (response.status_code)
        if response.status_code == 200:
            print ('ok1')
            self.dynamics(response.text)
        
        response = self.session.get(self.logined_url, headers=self.headers)
        if response.status_code == 200:
            print ('ok2')
            self.profile(response.text)
            
    def dynamics(self, html):
        selector = etree.HTML(html)
        dynamics = selector.xpath('//div[contains(@class, "news")]//div[contains(@class, "alert")]')
        for item in dynamics:
            dynamic = ' '.join(item.xpath('.//div[@class="title"]//text()')).strip()
            print(dynamic)
    
    def profile(self, html):
        selector = etree.HTML(html)
        name = selector.xpath('//input[@id="user_profile_name"]/@value')
        email = selector.xpath('//select[@id="user_profile_email"]/option[@value!=""]/text()')
        print(name, email)
        

if __name__ == "__main__":
    login = Login()
    login.login(email='386333667@qq.com', password='zhzh995=')

In [ ]:
import requests
from lxml import etree

class Login(object):
    def __init__(self):
        self.headers = {
            'Referer': 'https://github.com/',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
            'Host': 'github.com'
        }
        self.login_url = 'https://github.com/login'
        self.post_url = 'https://github.com/session'
        self.logined_url = 'https://github.com/settings/profile'
        self.session = requests.Session()
    
    def token(self):
        response = self.session.get(self.login_url, headers=self.headers)
        selector = etree.HTML(response.text)
        token = selector.xpath('//div//input[2]/@value')
        return token
    
    def login(self, email, password):
        post_data = {
            'commit': 'Sign in',
            'utf8': '✓',
            'authenticity_token': self.token()[0],
            'login': email,
            'password': password
        }
        response = self.session.post(self.post_url, data=post_data, headers=self.headers)
        if response.status_code == 200:
            self.dynamics(response.text)
        
        response = self.session.get(self.logined_url, headers=self.headers)
        if response.status_code == 200:
            self.profile(response.text)
    
    def dynamics(self, html):
        selector = etree.HTML(html)
        dynamics = selector.xpath('//div[contains(@class, "news")]//div[contains(@class, "alert")]')
        for item in dynamics:
            dynamic = ' '.join(item.xpath('.//div[@class="title"]//text()')).strip()
            print(dynamic)
    
    def profile(self, html):
        selector = etree.HTML(html)
        name = selector.xpath('//input[@id="user_profile_name"]/@value')[0]
        email = selector.xpath('//select[@id="user_profile_email"]/option[@value!=""]/text()')
        print(name, email)


if __name__ == "__main__":
    login = Login()
    login.login(email='386333667@qq.com', password='zhzh995=')

In [ ]:
from selenium import webdriver
browser = webdriver.Chrome()

In [ ]:
import math
math.isnan('nan')

In [ ]:
print(float(-1))

In [ ]:
with open('Documents/Codeforces/Muli4/output.txt') as fp:
    for expr in fp:
        if (expr != 'IMPOSSIBLE' and expr != None):
            x = eval(expr)
            print (x)

In [ ]:
ans = []

In [ ]:
for i in range(100000):
    for j in range(i*i+1, (i+1)*(i+1)):
        if (i*i*i*i % j == 0):
            ans.append(i)
            break
print (ans)

# Goverment Datas

In [320]:
from lxml import etree
import requests, re, time
from bs4 import BeautifulSoup
headers = {
    'User-Agent':'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'
}

## ShangWuBu

### China

In [2]:
def get_swb_china():
    print ('In {}, begin, date is {}'.format(get_swb_china.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))
        
    base_url = 'http://www.mofcom.gov.cn'
    response = requests.get(base_url + '/article/zhengcejd/', headers=headers)
    text = str(response.content, 'utf-8')
    html = etree.HTML(text)
    xpath = '//*[@id="wrap"]/div[2]/div/div[1]/div[2]/ul/li/a[1]/text()'
    titles = html.xpath(xpath)
    # print (titles)
    xpath = '//*[@id="wrap"]/div[2]/div/div[1]/div[2]/ul/li/a[1]/@href'
    links = html.xpath(xpath)
    # print (links)
    xpath = '//*[@id="wrap"]/div[2]/div/div[1]/div[2]/ul/li/span/text()'
    dates = html.xpath(xpath)
    # print (dates)

    data_china = [ (t, l, d, 'china') for (t, l, d) in zip(titles, links, dates) ]

    response = requests.get(base_url + '/article/b/', headers=headers)
    text = str(response.content, 'utf-8')
    html = etree.HTML(text)
    xpath = '//*[@id="wrap"]/div[2]/div/div[1]/div[2]/ul/li/a/text()'
    titles = html.xpath(xpath)
    xpath = '//*[@id="wrap"]/div[2]/div/div[1]/div[2]/ul/li/a/@href'
    links = html.xpath(xpath)
    links = [ base_url + str(x) for x in links]
    xpath = '//*[@id="wrap"]/div[2]/div/div[1]/div[2]/ul/li/span/text()'
    dates = html.xpath(xpath)

    data_china = data_china + [ (t, l, d, 'china') for t, l, d in zip(titles, links, dates) ]

    dict_china = [{'pos':x[3], 'title': x[0], 'link': x[1], 'date': x[2]} for x in data_china]
  
    print (len(dict_china))

    print ('In {}, end, date is {}'.format(get_swb_china.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    return dict_china

### JiangSu

In [3]:
def get_swb_jiangsu():
    
    print ('In {}, begin, date is {}'.format(get_swb_jiangsu.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    base_url = 'http://swt.jiangsu.gov.cn'
    response = requests.get(base_url + '/col/col12660/index.html', headers=headers)
    text = str(response.content, 'utf-8')
    # print (text)
    regex = re.compile('<a href="(.*)" target="_blank">(.*)</a><span style=".*"> \((.*)\)</span>')

    data_jiangsu = [(x[1], base_url+x[0], x[2].replace('/', '-'), 'jiangsu') for x in re.findall(pattern=regex, string=text)]
    
    dict_jiangsu = [{'pos':x[3], 'title': x[0], 'link': x[1], 'date': x[2]} for x in data_jiangsu]
    
    print (len(dict_jiangsu))
    
    print ('In {}, end, date is {}'.format(get_swb_jiangsu.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    return dict_jiangsu

## ZheJiang

In [4]:
def get_swb_zhejiang():
    
    print ('In {}, begin, date is {}'.format(get_swb_zhejiang.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    base_url = 'http://zhejiang.mofcom.gov.cn'
    response = requests.get(base_url + '/article/sjtongzhigg', headers=headers)
    text = str(response.content, 'utf-8')
    html = etree.HTML(text)
    xpath = '//*[@id="main"]/div[2]/div/ul/li/a/text()'
    titles = html.xpath(xpath)
    xpath = '//*[@id="main"]/div[2]/div/ul/li/a/@href'
    links = html.xpath(xpath)
    links = [ base_url + str(x) for x in links]
    xpath = '//*[@id="main"]/div[2]/div/ul/li/span/text()'
    dates = html.xpath(xpath)
    dates = [x.split()[0] for x in dates]

    data_zhejiang = [ (t, l, d, 'zhejiang') for t, l, d in zip(titles, links, dates) ]
    
    dict_zhejiang = [{'pos':x[3], 'title': x[0], 'link': x[1], 'date': x[2]} for x in data_zhejiang]
    
    print (len(dict_zhejiang))

    print ('In {}, end, date is {}'.format(get_swb_zhejiang.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    return dict_zhejiang

## GuangDong

In [5]:
def get_swb_guangdong():
    
    print ('In {}, begin, date is {}'.format(get_swb_guangdong.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    base_url = 'http://www.gdcom.gov.cn/zwgk/zcwj/'
    response = requests.get(base_url, headers=headers)
    
    if (response.status_code != 200):
        return None

    text = str(response.content, 'utf-8')
    html = etree.HTML(text)
    xpath = '/html/body/div[2]/div/div[2]/ul/li/a/text()'
    titles = html.xpath(xpath)
    xpath = '/html/body/div[2]/div/div[2]/ul/li/a/@href'
    links = html.xpath(xpath)
    tlinks = []
    for x in links:
        x = str(x)
        if x != None and x[:2] == './':
            x = base_url + x[2:]
        tlinks.append(x)
    links = tlinks
    
    xpath = '/html/body/div[2]/div/div[2]/ul/li/span/text()'
    dates = html.xpath(xpath)
    dates = [x.replace(' ', '') for x in dates]
    
    data_guangdong = [ (t, l, d, 'guangdong') for t, l, d in zip(titles, links, dates) ]
        
    dict_guangdong = [{'pos':x[3], 'title': x[0], 'link': x[1], 'date': x[2]} for x in data_guangdong]
    
    print (len(dict_guangdong))

    print ('In {}, end, date is {}'.format(get_swb_guangdong.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    return dict_guangdong

## Test Mysql

## Mysql Template

In [14]:
import pymysql
# db = pymysql.connect(host='119.29.190.115', user='impulse', password='njuacmicpc', port=3306)
# cursor = db.cursor()

In [7]:
def create_db(cursor, name):
    sql = 'create database {name} default character set utf8'.format(name=name)
    cursor.execute(sql)

In [321]:
# # Create table!!!
# import pymysql
# db = pymysql.connect(host='119.29.190.115', user='impulse', password='njuacmicpc', port=3306, db='gov')
# cursor = db.cursor()
# sql = 'create table if not exists zscqj (pos VARCHAR(32), title VARCHAR(512), link VARCHAR(512), date VARCHAR(64), PRIMARY KEY (link))'
# cursor.execute(sql)
# db.close()

In [15]:
def sql_insert(db, cursor, table, data):
    keys = ', '.join(data.keys())
    values = ', '.join(['%s'] * len(data))
    sql = 'insert into {table}({keys}) values ({values}) on duplicate key update'.format(table=table, 
            keys=keys, values=values)
    update = ','.join([" {key} = %s".format(key=key) for key in data])
    sql += update
    #print (sql)
    try:
        if cursor.execute(sql, tuple(data.values())*2):
            print('Successful')
            db.commit()
        else:
            print ('Nothing to do')
    except:
        print ('Failed')
        db.rollback()

In [16]:
def get_swb():
    db = pymysql.connect(host='119.29.190.115', user='impulse', password='njuacmicpc', port=3306, db='gov')
    cursor = db.cursor()
    for dic in get_swb_china():
        sql_insert(db=db,cursor=cursor,data=dic,table='swb')
    for dic in get_swb_jiangsu():
        sql_insert(db=db,cursor=cursor,data=dic,table='swb')
    for dic in get_swb_guangdong():
        sql_insert(db=db,cursor=cursor,data=dic,table='swb')
    for dic in get_swb_zhejiang():
        sql_insert(db=db,cursor=cursor,data=dic,table='swb')
    db.close()

## ShangWuBuWaiMaoSi

In [88]:
def get_wms_china():
    
    print ('In {}, begin, date is {}'.format(get_wms_china.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    base_url = 'http://wms.mofcom.gov.cn'
    response = requests.get(base_url + '/article/zcfb/ax/', headers=headers)
        
    text = str(response.content, 'utf-8')
    html = etree.HTML(text)
    xpath = '//*[@id="leftList"]/div[2]/ul/li/a[1]/text()'
    titles = html.xpath(xpath)
    # print (titles)
    xpath = '//*[@id="leftList"]/div[2]/ul/li/a[1]/@href'
    links = html.xpath(xpath)
    links = [ base_url + str(x) for x in links]
    # print (links)
    xpath = '//*[@id="leftList"]/div[2]/ul/li/span/text()'
    dates = html.xpath(xpath)
    dates = [x.split()[0] for x in dates]
    # print (dates)

    data_china = [ (t, l, d, 'china') for (t, l, d) in zip(titles, links, dates) ]

    response = requests.get(base_url + '/article/zcfb/g/', headers=headers)
    
    text = str(response.content, 'utf-8')
    html = etree.HTML(text)
    xpath = '//*[@id="leftList"]/div[2]/ul/li/a[1]/text()'
    titles = html.xpath(xpath)
    xpath = '//*[@id="leftList"]/div[2]/ul/li/a[1]/@href'
    links = html.xpath(xpath)
    links = [ base_url + str(x) for x in links]
    xpath = '//*[@id="leftList"]/div[2]/ul/li/span/text()'
    dates = html.xpath(xpath)
    dates = [x.split()[0] for x in dates]
    
    data_china = data_china + [ (t, l, d, 'china') for t, l, d in zip(titles, links, dates) ]

    dict_china = [{'pos':x[3], 'title': x[0], 'link': x[1], 'date': x[2]} for x in data_china]
  
    print (len(dict_china))

    print ('In {}, end, date is {}'.format(get_wms_china.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    return dict_china

In [89]:
def get_wms_jiangsu():

    print ('In {}, begin, date is {}'.format(get_wms_jiangsu.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    base_url = 'http://swt.jiangsu.gov.cn'
    
    response = requests.get(base_url + '/col/col57691/index.html', headers=headers)
    text = str(response.content, 'utf-8')
    regex = re.compile('<a href="(.*)" target="_blank" .*title="(.*)">.*</a><span.*> \((.*)\)</span>')
    data_jiangsu = [(x[1], base_url+x[0], x[2].replace('/', '-'), 'jiangsu') for x in re.findall(pattern=regex, string=text)]
    
    response = requests.get(base_url + '/col/col57692/index.html', headers=headers)
    text = str(response.content, 'utf-8')
    regex = re.compile('<a href="(.*)" target="_blank" .*title="(.*)">.*</a><span.*> \((.*)\)</span>')
    data_jiangsu += [(x[1], base_url+x[0], x[2].replace('/', '-'), 'jiangsu') for x in re.findall(pattern=regex, string=text)]
    
    dict_jiangsu = [{'pos':x[3], 'title': x[0], 'link': x[1], 'date': x[2]} for x in data_jiangsu]
    
    print (len(dict_jiangsu))

    print ('In {}, end, date is {}'.format(get_wms_jiangsu.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    return dict_jiangsu

In [92]:
def get_wms_zhejiang():

    print ('In {}, begin, date is {}'.format(get_wms_zhejiang.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    base_url = 'http://www.zcom.gov.cn'
    
    response = requests.get(base_url + '/col/col1385815/index.html', headers=headers)
    text = str(response.content, 'utf-8')

    html = etree.HTML(text)
    xpath = '//*[@id="con_two_2"]/div[position()>1]/a[1]/text()'
    titles = html.xpath(xpath)  
    xpath = '//*[@id="con_two_2"]/div[position()>1]/a[1]/@href'
    links = html.xpath(xpath)
    links = [ base_url + str(x) for x in links]
    
    xpath = '//*[@id="con_two_2"]/div[position()>1]/a[1]/span/text()'
    dates = html.xpath(xpath)
    data_zhejiang = [ (t, l, d, 'zhejiang') for t, l, d in zip(titles, links, dates) ]

    ################
    
    xpath = '//*[@id="con_three_1"]/div[position()>1]/a[1]/text()'
    titles = html.xpath(xpath)  
    xpath = '//*[@id="con_three_1"]/div[position()>1]/a[1]/@href'
    links = html.xpath(xpath)
    links = [ base_url + str(x) for x in links]
    
    xpath = '//*[@id="con_three_1"]/div[position()>1]/a[1]/span/text()'
    dates = html.xpath(xpath)
    data_zhejiang += [ (t, l, d, 'zhejiang') for t, l, d in zip(titles, links, dates) ]
    
    dict_zhejiang = [{'pos':x[3], 'title': x[0], 'link': x[1], 'date': x[2]} for x in data_zhejiang]
  
    print (len(dict_zhejiang))

    print ('In {}, end, date is {}'.format(get_wms_zhejiang.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    return dict_zhejiang

In [91]:
def get_wms_guangdong():
    
    print ('In {}, begin, date is {}'.format(get_wms_guangdong.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    base_url = 'http://go.gdcom.gov.cn'
    
    response = requests.get(base_url + '/article.php?typeid=9', headers=headers)
    text = str(response.content, 'utf-8')

    html = etree.HTML(text)
    xpath = '/html/body/div[3]/div/div[2]/div/div[2]/div[2]/ul/li/h6/a[1]/text()'
    titles = html.xpath(xpath)  
    xpath = '/html/body/div[3]/div/div[2]/div/div[2]/div[2]/ul/li/h6/a[1]/@href'
    links = html.xpath(xpath)
    links = [ base_url + '/' + str(x) for x in links]
    
    xpath = '/html/body/div[3]/div/div[2]/div/div[2]/div[2]/ul/li/h6/small/text()'
    dates = html.xpath(xpath)
    data_guangdong = [ (t, l, d, 'guangdong') for t, l, d in zip(titles, links, dates) ]

    dict_guangdong = [{'pos':x[3], 'title': x[0], 'link': x[1], 'date': x[2]} for x in data_guangdong]
  
    print (len(dict_guangdong))

    print ('In {}, end, date is {}'.format(get_wms_guangdong.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    return dict_guangdong

In [93]:
def get_wms():
    db = pymysql.connect(host='119.29.190.115', user='impulse', password='njuacmicpc', port=3306, db='gov')
    cursor = db.cursor()
    for dic in get_wms_china():
        sql_insert(db=db,cursor=cursor,data=dic,table='wms')
    for dic in get_wms_jiangsu():
        sql_insert(db=db,cursor=cursor,data=dic,table='wms')
    for dic in get_wms_guangdong():
        sql_insert(db=db,cursor=cursor,data=dic,table='wms')
    for dic in get_wms_zhejiang():
        sql_insert(db=db,cursor=cursor,data=dic,table='wms')
    db.close()

## ZhiShiChanQuanJu

In [99]:
def get_zscqj_china():
    
    print ('In {}, begin, date is {}'.format(get_zscqj_china.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    base_url = 'http://www.sipo.gov.cn'
    
    response = requests.get(base_url + '/gwywj/index.htm', headers=headers)     
    text = str(response.content, 'utf-8')
    html = etree.HTML(text)
    xpath = '/html/body/div/div/div/div[4]/div/div[2]/ul/li/a[1]/text()'
    titles = html.xpath(xpath)
    # print (titles)
    xpath = '/html/body/div/div/div/div[4]/div/div[2]/ul/li/a[1]/@href'
    links = html.xpath(xpath)
    # print (links)
    xpath = '/html/body/div/div/div/div[4]/div/div[2]/ul/li/span/text()'
    dates = html.xpath(xpath)
    # print (dates)
    data_china = [ (t, l, d, 'china') for (t, l, d) in zip(titles, links, dates) ]

    response = requests.get(base_url + '/dtxx/index.htm', headers=headers)
    text = str(response.content, 'utf-8')
    html = etree.HTML(text)
    xpath = '/html/body/div/div/div/div[4]/div/div[2]/ul/li/a[1]/text()'
    titles = html.xpath(xpath)
    # print (titles)
    xpath = '/html/body/div/div/div/div[4]/div/div[2]/ul/li/a[1]/@href'
    links = html.xpath(xpath)
    ## !!!!!
    links = [ base_url + '/dtxx/' + str(x) for x in links] 
    # print (links)
    xpath = '/html/body/div/div/div/div[4]/div/div[2]/ul/li/span/text()'
    dates = html.xpath(xpath)
    # print (dates) 
    data_china = data_china + [ (t, l, d, 'china') for t, l, d in zip(titles, links, dates) ]

    response = requests.get(base_url + '/zfgg/index.htm', headers=headers)
    text = str(response.content, 'utf-8')
    html = etree.HTML(text)
    xpath = '/html/body/div/div/div/div[4]/div/div[2]/ul/li/a[1]/text()'
    titles = html.xpath(xpath)
    # print (titles)
    xpath = '/html/body/div/div/div/div[4]/div/div[2]/ul/li/a[1]/@href'
    links = html.xpath(xpath)
    ## !!!!!
    links = [ base_url + '/zfgg/' + str(x) for x in links]
    # print (links)
    xpath = '/html/body/div/div/div/div[4]/div/div[2]/ul/li/span/text()'
    dates = html.xpath(xpath)
    # print (dates) 
    data_china = data_china + [ (t, l, d, 'china') for t, l, d in zip(titles, links, dates) ]
    
    response = requests.get(base_url + '/gztz/index.htm', headers=headers)
    text = str(response.content, 'utf-8')
    html = etree.HTML(text)
    xpath = '/html/body/div/div/div/div[4]/div/div[2]/ul/li/a[1]/text()'
    titles = html.xpath(xpath)
    # print (titles)
    xpath = '/html/body/div/div/div/div[4]/div/div[2]/ul/li/a[1]/@href'
    links = html.xpath(xpath)
    ## !!!!!
    links = [ base_url + '/gztz/' + str(x) for x in links]
    # print (links)
    xpath = '/html/body/div/div/div/div[4]/div/div[2]/ul/li/span/text()'
    dates = html.xpath(xpath)
    # print (dates) 
    data_china = data_china + [ (t, l, d, 'china') for t, l, d in zip(titles, links, dates) ]
    
    
    dict_china = [{'pos':x[3], 'title': x[0], 'link': x[1], 'date': x[2]} for x in data_china]
  
    print (len(dict_china))

    print ('In {}, end, date is {}'.format(get_zscqj_china.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    return dict_china

In [112]:
def get_zscqj_jiangsu():

    print ('In {}, begin, date is {}'.format(get_zscqj_jiangsu.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    base_url = 'http://jsip.jiangsu.gov.cn'
    
    response = requests.get(base_url + '/col/col3300/index.html', headers=headers)
    text = str(response.content, 'utf-8')
#     print (text)
    regex = re.compile('<a href="(.*)" TARGET="_blank" >.*<div class="text">(.*)</div>.*<div class="text-date">(.*)</div>.*</a>')
    data_jiangsu = [(x[1], base_url+x[0], x[2], 'jiangsu') for x in re.findall(pattern=regex, string=text)]
    
    response = requests.get(base_url + '/col/col3252/index.html', headers=headers)
    text = str(response.content, 'utf-8')
    regex = re.compile('<a href="(.*)" TARGET="_blank" >.*<div class="text">(.*)</div>.*<div class="text-date">(.*)</div>.*</a>')
    data_jiangsu += [(x[1], base_url+x[0], x[2], 'jiangsu') for x in re.findall(pattern=regex, string=text)]
    
    dict_jiangsu = [{'pos':x[3], 'title': x[0], 'link': x[1], 'date': x[2]} for x in data_jiangsu]
    
    print (len(dict_jiangsu))

    print ('In {}, end, date is {}'.format(get_zscqj_jiangsu.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    return dict_jiangsu

In [281]:
def get_zscqj_zhejiang():

    print ('In {}, begin, date is {}'.format(get_zscqj_zhejiang.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    base_url = 'http://www.zjpat.gov.cn'
    
    response = requests.get(base_url + '/interIndex.do?method=list22&dir=/zjszscqj/tzgg', headers=headers)
    text = str(response.content, 'utf-8').replace('\r\n', '')
    text = re.sub(' +', ' ', text)

    regex = re.compile('<a href="(.*?)" title="(.*?)" target="_blank" class="color_01">.*?</a> </td> <td.*?> \((.*?)\) </td> </tr>')
    data_zhejiang = [(x[1], base_url+'/'+x[0], x[2], 'zhejiang') for x in re.findall(pattern=regex, string=text)[1:]]
    
    response = requests.get(base_url + '/interIndex.do?method=list2&dir=/zjszscqj/xwdt/sxdt', headers=headers)
    text = str(response.content, 'utf-8').replace('\r\n', '')
    text = re.sub(' +', ' ', text)
    regex = re.compile('<a href="(.*?)" title="(.*?)" target="_blank" class="color_01">.*?</a> </td> <td.*?> \((.*?)\) </td> </tr>')
    data_zhejiang += [(x[1], base_url+'/'+x[0], x[2], 'zhejiang') for x in re.findall(pattern=regex, string=text)[1:]]
    
    dict_zhejiang = [{'pos':x[3], 'title': x[0], 'link': x[1], 'date': x[2]} for x in data_zhejiang]
    
    print (len(dict_zhejiang))
    
    print ('In {}, end, date is {}'.format(get_zscqj_zhejiang.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    return dict_zhejiang

### Very/interIndex.do?method=list2&dir=/zjszscqj/xwdt/sxdt Important to match \r\n...
### re.findall(pattern='<table>([\s\S]*?)<\/table>', string=text)

In [317]:
def get_zscqj_guangdong():

    print ('In {}, begin, date is {}'.format(get_zscqj_guangdong.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    base_url = 'http://www.gdipo.gov.cn'
    
    response = requests.get(base_url + '/gdipo/gdipodt/list.shtml', headers=headers)
    text = str(response.content, 'utf-8').replace('\r\n', '')
    text = re.sub(' +', ' ', text)
    regex = re.compile('<li> <a href="(.*?)" target="_blank" >(.*?)</a> <div class="time_div"> <span class="time">(.*?)</span> </div> </li>')
    data_guangdong = [(x[1], base_url+x[0], x[2], 'guangdong') for x in re.findall(pattern=regex, string=text)]

    response = requests.get(base_url + '/gdipo/tzgg/list.shtml', headers=headers)
    text = str(response.content, 'utf-8').replace('\r\n', '')
    text = re.sub(' +', ' ', text)
    regex = re.compile('<li> <a href="(.*?)" target="_blank" >(.*?)</a> <div class="time_div"> <span class="time">(.*?)</span> </div> </li>')
    data_guangdong += [(x[1], base_url+x[0], x[2], 'guangdong') for x in re.findall(pattern=regex, string=text)]
   
    dict_guangdong = [{'pos':x[3], 'title': x[0], 'link': x[1], 'date': x[2]} for x in data_guangdong]
    
    print (len(dict_guangdong))
    
    print ('In {}, end, date is {}'.format(get_zscqj_guangdong.__name__, time.strftime('%Y-%m-%d-%H:%M', time.localtime(time.time()))))

    return dict_guangdong

### Very/interIndex.do?method=list2&dir=/zjszscqj/xwdt/sxdt Important to match \r\n...
### re.findall(pattern='<table>([\s\S]*?)<\/table>', string=text)

In [326]:
def get_zscqj():
    db = pymysql.connect(host='119.29.190.115', user='impulse', password='njuacmicpc', port=3306, db='gov')
    cursor = db.cursor()
    for dic in get_zscqj_china():
        sql_insert(db=db,cursor=cursor,data=dic,table='zscqj')
    for dic in get_zscqj_jiangsu():
        sql_insert(db=db,cursor=cursor,data=dic,table='zscqj')
    for dic in get_zscqj_guangdong():
        sql_insert(db=db,cursor=cursor,data=dic,table='zscqj')
    for dic in get_zscqj_zhejiang():
        sql_insert(db=db,cursor=cursor,data=dic,table='zscqj')
    db.close()